In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

# Load the Dataset

In [3]:
df = pd.read_csv('fifa21 raw data v2.csv')

C:\Users\Burhan\AppData\Local\Temp\ipykernel_13428\235989469.py:1: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('fifa21 raw data v2.csv')


In [4]:
df.head(5)


,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,158023,L. Messi,Lionel Messi,https://cdn.sofifa.com/players/158/023/21_60.png,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,33,93,93,\n\n\n\nFC Barcelona,...,Medium,Low,5 ★,85,92,91,95,38,65,771
1,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,https://cdn.sofifa.com/players/020/801/21_60.png,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,35,92,92,\n\n\n\nJuventus,...,High,Low,5 ★,89,93,81,89,35,77,562
2,200389,J. Oblak,Jan Oblak,https://cdn.sofifa.com/players/200/389/21_60.png,http://sofifa.com/player/200389/jan-oblak/210006/,Slovenia,27,91,93,\n\n\n\nAtlético Madrid,...,Medium,Medium,3 ★,87,92,78,90,52,90,150
3,192985,K. De Bruyne,Kevin De Bruyne,https://cdn.sofifa.com/players/192/985/21_60.png,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,29,91,91,\n\n\n\nManchester City,...,High,High,4 ★,76,86,93,88,64,78,207
4,190871,Neymar Jr,Neymar da Silva Santos Jr.,https://cdn.sofifa.com/players/190/871/21_60.png,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,28,91,91,\n\n\n\nParis Saint-Germain,...,High,Medium,5 ★,91,85,86,94,36,59,595


# Explore the Data

In [6]:
# Count of missing values per column
missing_values = df.isnull().sum().sort_values(ascending=False)
missing_values[missing_values > 0]

Loan Date End    17966
Hits              2595
dtype: int64

In [7]:
# Data types of each column
df.dtypes

ID            int64
Name         object
LongName     object
photoUrl     object
playerUrl    object
              ...  
PAS           int64
DRI           int64
DEF           int64
PHY           int64
Hits         object
Length: 77, dtype: object

In [8]:
# Sample values for columns with known format inconsistencies
print("Unique Height Formats:", df['Height'].unique()[:10])
print("Unique Weight Formats:", df['Weight'].unique()[:10])
print("Sample Wages:", df['Wage'].unique()[:10])
print("Sample Values:", df['Value'].unique()[:10])

Unique Height Formats: ['170cm' '187cm' '188cm' '181cm' '175cm' '184cm' '191cm' '178cm' '193cm'
 '185cm']
Unique Weight Formats: ['72kg' '83kg' '87kg' '70kg' '68kg' '80kg' '71kg' '91kg' '73kg' '85kg']
Sample Wages: ['€560K' '€220K' '€125K' '€370K' '€270K' '€240K' '€250K' '€160K' '€260K'
 '€210K']
Sample Values: ['€103.5M' '€63M' '€120M' '€129M' '€132M' '€111M' '€120.5M' '€102M'
 '€185.5M' '€110M']


In [9]:
# Check for duplicate rows
duplicate_rows = df.duplicated().sum()
print("Duplicate Rows:", duplicate_rows)

# List of columns
print("Total Columns:", len(df.columns))
print("Column Names:\n", df.columns.tolist())

Duplicate Rows: 0
Total Columns: 77
Column Names:
 ['ID', 'Name', 'LongName', 'photoUrl', 'playerUrl', 'Nationality', 'Age', '↓OVA', 'POT', 'Club', 'Contract', 'Positions', 'Height', 'Weight', 'Preferred Foot', 'BOV', 'Best Position', 'Joined', 'Loan Date End', 'Value', 'Wage', 'Release Clause', 'Attacking', 'Crossing', 'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill', 'Dribbling', 'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control', 'Movement', 'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance', 'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots', 'Mentality', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Defending', 'Marking', 'Standing Tackle', 'Sliding Tackle', 'Goalkeeping', 'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Total Stats', 'Base Stats', 'W/F', 'SM', 'A/W', 'D/W', 'IR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'Hits']


# Handle Missing Values

In [11]:
# Make a copy to clean
df_cleaned = df.copy()

# Impute numerical columns with median
num_cols = df_cleaned.select_dtypes(include=['float64', 'int64']).columns
df_cleaned[num_cols] = df_cleaned[num_cols].apply(lambda x: x.fillna(x.median()))

# Fill missing categorical values with "Unknown"
cat_cols = df_cleaned.select_dtypes(include='object').columns
df_cleaned[cat_cols] = df_cleaned[cat_cols].fillna("Unknown")

# Convert height to cm

In [13]:
def convert_height(height_str):
    try:
        feet, inches = height_str.split("'")
        return round((int(feet) * 30.48) + (int(inches.strip('"')) * 2.54), 2)
    except:
        return None

df_cleaned['Height(cm)'] = df_cleaned['Height'].apply(convert_height)


# Convert weight from lbs to kg

In [15]:
df_cleaned['Weight(kg)'] = df_cleaned['Weight'].str.replace('lbs', '', regex=False).str.strip()
df_cleaned['Weight(kg)'] = pd.to_numeric(df_cleaned['Weight(kg)'], errors='coerce')

# Convert Wage, Value, Release Clause to integers

In [17]:
def parse_money(val):
    if pd.isna(val) or val == 'Unknown':
        return None
    val = val.replace('€', '').replace('M', '000000').replace('K', '000')
    try:
        return int(float(val))
    except:
        return None

for col in ['Wage', 'Value', 'Release Clause']:
    df_cleaned[col + '(€)'] = df_cleaned[col].apply(parse_money)
def parse_money(val):
    if pd.isna(val) or val == 'Unknown':
        return None
    val = val.replace('€', '').replace('M', '000000').replace('K', '000')
    try:
        return int(float(val))
    except:
        return None

for col in ['Wage', 'Value', 'Release Clause']:
    df_cleaned[col + '(€)'] = df_cleaned[col].apply(parse_money)


# Convert contract year to datetime

In [19]:
# Clean up column name spacing (good practice)
df_cleaned.columns = df_cleaned.columns.str.strip()

# Convert 'Contract' column to datetime (assuming format like "2023" or "Jun 30, 2023")
df_cleaned['Contract'] = pd.to_datetime(df_cleaned['Contract'], errors='coerce').dt.date

# Fill any missing dates with a default
df_cleaned['Contract'] = df_cleaned['Contract'].fillna(pd.to_datetime("2021-01-01").date())


C:\Users\Burhan\AppData\Local\Temp\ipykernel_13428\1356118946.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_cleaned['Contract'] = pd.to_datetime(df_cleaned['Contract'], errors='coerce').dt.date


# Correct Data Types

In [21]:
# Convert relevant columns to numeric
df_cleaned['Age'] = pd.to_numeric(df_cleaned['Age'], errors='coerce', downcast='integer')
df_cleaned['Wage(€)'] = pd.to_numeric(df_cleaned['Wage(€)'], errors='coerce')
df_cleaned['Value(€)'] = pd.to_numeric(df_cleaned['Value(€)'], errors='coerce')

# Ensure categorical columns are strings
cat_cols = ['Name', 'Nationality', 'Club', 'Preferred Foot', 'Positions']
for col in cat_cols:
    df_cleaned[col] = df_cleaned[col].astype(str)

# Remove Redundancies

In [23]:
# Drop irrelevant/redundant columns
redundant_cols = ['ID', 'Photo', 'Flag', 'Club Logo', 'Work Rate', 'Real Face',
                  'Loaned From', 'Body Type', 'Height', 'Weight', 'Wage', 'Value', 'Release Clause']
df_cleaned = df_cleaned.drop(columns=redundant_cols, errors='ignore')

# Normalize text: Standardize known club name variations
df_cleaned['Club'] = df_cleaned['Club'].replace({
    'FC Bayern München': 'Bayern Munich',
    'FC Barcelona': 'Barcelona',
    'Real Madrid CF': 'Real Madrid'
    # Add more mappings if needed
})

# Player Role from Position

In [25]:
def assign_role(position):
    if position in ['ST', 'CF', 'LW', 'RW']:
        return 'Forward'
    elif position in ['CM', 'CDM', 'CAM', 'RM', 'LM']:
        return 'Midfielder'
    elif position in ['CB', 'LB', 'RB', 'LWB', 'RWB']:
        return 'Defender'
    elif position == 'GK':
        return 'Goalkeeper'
    else:
        return 'Other'

df_cleaned['Player Role'] = df_cleaned['Positions'].apply(assign_role)

In [26]:
df_cleaned.to_csv('fifa21_cleaned.csv', index=False)

# Analyze cleaned data

In [28]:
df2 = pd.read_csv('fifa21_cleaned.csv')

In [29]:
df2.head(5)

,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,Contract,...,DRI,DEF,PHY,Hits,Height(cm),Weight(kg),Wage(€),Value(€),Release Clause(€),Player Role
0,L. Messi,Lionel Messi,https://cdn.sofifa.com/players/158/023/21_60.png,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,33,93,93,\n\n\n\nFC Barcelona,2021-01-01,...,95,38,65,771,NaN,NaN,560000,103,138,Other
1,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,https://cdn.sofifa.com/players/020/801/21_60.png,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,35,92,92,\n\n\n\nJuventus,2021-01-01,...,89,35,77,562,NaN,NaN,220000,63000000,75,Other
2,J. Oblak,Jan Oblak,https://cdn.sofifa.com/players/200/389/21_60.png,http://sofifa.com/player/200389/jan-oblak/210006/,Slovenia,27,91,93,\n\n\n\nAtlético Madrid,2021-01-01,...,90,52,90,150,NaN,NaN,125000,120000000,159,Goalkeeper
3,K. De Bruyne,Kevin De Bruyne,https://cdn.sofifa.com/players/192/985/21_60.png,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,29,91,91,\n\n\n\nManchester City,2021-01-01,...,88,64,78,207,NaN,NaN,370000,129000000,161000000,Other
4,Neymar Jr,Neymar da Silva Santos Jr.,https://cdn.sofifa.com/players/190/871/21_60.png,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,28,91,91,\n\n\n\nParis Saint-Germain,2021-01-01,...,94,36,59,595,NaN,NaN,270000,132000000,166,Other


In [30]:
from sqlalchemy import create_engine

# Load cleaned dataset
df = pd.read_csv("fifa21_cleaned.csv")

# Create SQLite connection
engine = create_engine('sqlite:///fifa_star_schema.db')


# Create Dimension Tables

# Player Dimension

In [33]:
# ---------------------- Player Dimension ----------------------
dim_player = df[['Name', 'Age', 'Height(cm)', 'Weight(kg)']].drop_duplicates().reset_index(drop=True)
dim_player['PlayerID'] = dim_player.index + 1
dim_player = dim_player[['PlayerID', 'Name', 'Age', 'Height(cm)', 'Weight(kg)']]

# Time Dimension

In [35]:
# ----------------------  ----------------------
df['Contract'] = pd.to_datetime(df['Contract'], errors='coerce')
dim_time = df[['Contract']].dropna().drop_duplicates().reset_index(drop=True)
dim_time['TimeID'] = dim_time.index + 1
dim_time['Year'] = dim_time['Contract'].dt.year
dim_time['Month'] = dim_time['Contract'].dt.month
dim_time = dim_time[['TimeID', 'Year', 'Month', 'Contract']]

# Club Dimension

In [37]:
# ---------------------- Club Dimension ----------------------
dim_club = df[['Club']].drop_duplicates().reset_index(drop=True)
dim_club['ClubID'] = dim_club.index + 1
dim_club['League'] = 'Unknown'  # Placeholder (your dataset doesn't include league_name)
dim_club = dim_club[['ClubID', 'Club', 'League']]

# Nationality Dimension

In [39]:
# ---------------------- Nationality Dimension ----------------------
dim_nationality = df[['Nationality']].drop_duplicates().reset_index(drop=True)
dim_nationality['NationalityID'] = dim_nationality.index + 1
dim_nationality = dim_nationality[['NationalityID', 'Nationality']]

# Position Dimension 

In [41]:
# ---------------------- Position Dimension ----------------------
position_role_map = {
    'ST': 'Forward', 'CF': 'Forward', 'LW': 'Forward', 'RW': 'Forward',
    'CM': 'Midfielder', 'CDM': 'Midfielder', 'CAM': 'Midfielder', 'RM': 'Midfielder', 'LM': 'Midfielder',
    'CB': 'Defender', 'LB': 'Defender', 'RB': 'Defender', 'LWB': 'Defender', 'RWB': 'Defender',
    'GK': 'Goalkeeper'
}
df['PrimaryPosition'] = df['Positions'].apply(lambda x: x.split(',')[0].strip())
df['Role'] = df['PrimaryPosition'].map(position_role_map).fillna('Unknown')

dim_position = df[['PrimaryPosition', 'Role']].drop_duplicates().reset_index(drop=True)
dim_position['PositionID'] = dim_position.index + 1
dim_position = dim_position[['PositionID', 'PrimaryPosition', 'Role']]

# Fact Table 

In [43]:
# ---------------------- Fact Table ----------------------
# Merge to get foreign keys
df_fact = df.merge(dim_player, on=['Name', 'Age', 'Height(cm)', 'Weight(kg)']) \
            .merge(dim_time, on='Contract') \
            .merge(dim_club, on='Club') \
            .merge(dim_nationality, on='Nationality') \
            .merge(dim_position, left_on='PrimaryPosition', right_on='PrimaryPosition')

df_fact['Wage'] = df_fact['Wage(€)'].replace('[\€,]', '', regex=True).replace('K', '000', regex=True).astype(str).str.replace(r'[^\d]', '', regex=True).astype(float)

fact_table = df_fact[['PlayerID', 'TimeID', 'ClubID', 'NationalityID', 'PositionID', '↓OVA', 'Wage']]
fact_table = fact_table.rename(columns={'↓OVA': 'OverallRating', 'Wage': 'Wages'})

# Star Schema Diagram

In [45]:
pip install rich

Note: you may need to restart the kernel to use updated packages.


In [46]:
from rich.console import Console
from rich.text import Text

console = Console()

schema_diagram = Text("""
                   ┌──────────────┐
                   │  TimeDim     │
                   │  TimeID (PK) │
                   │  Year        │
                   │  Month       │
                   └─────┬────────┘
                         │
 ┌──────────────┐   ┌────▼─────┐   ┌──────────────┐   ┌─────────────────┐   ┌──────────────┐
 │ PlayerDim    │   │ Fact     │   │ ClubDim      │   │ NationalityDim  │   │ PositionDim  │
 │ PlayerID (PK)│◄──▶ Table    │◄──▶ ClubID (PK)  │◄──▶ NationalityID  │◄──▶ PositionID   │
 │ Name         │   │ PlayerID │   │ ClubName     │   │ Country         │   │ Position     │
 │ Age          │   │ TimeID   │   │ League       │   └─────────────────┘   │ Role         │
 │ Height       │   │ ClubID   │   └──────────────┘                        └──────────────┘
 │ Weight       │   │ NatID    │
 └──────────────┘   │ PosID    │
                    │ Rating   │
                    │ Wages    │
                    └──────────┘
""")

console.print(schema_diagram)


┌──────────────┐
                   │  TimeDim     │
                   │  TimeID (PK) │
                   │  Year        │
                   │  Month       │
                   └─────┬────────┘
                         │
 ┌──────────────┐   ┌────▼─────┐   ┌──────────────┐   ┌─────────────────┐   ┌──────────────┐
 │ PlayerDim    │   │ Fact     │   │ ClubDim      │   │ NationalityDim  │   │ PositionDim  │
 │ PlayerID (PK)│◄──▶ Table    │◄──▶ ClubID (PK)  │◄──▶ NationalityID  │◄──▶ PositionID   │
 │ Name         │   │ PlayerID │   │ ClubName     │   │ Country         │   │ Position     │
 │ Age          │   │ TimeID   │   │ League       │   └─────────────────┘   │ Role         │
 │ Height       │   │ ClubID   │   └──────────────┘                        └──────────────┘
 │ Weight       │   │ NatID    │
 └──────────────┘   │ PosID    │
                    │ Rating   │
                    │ Wages    │
                    └──────────┘

In [47]:
import sqlite3

# Connect to SQLite (creates FIFA21_DWH.db if it doesn't exist)
conn = sqlite3.connect('fifa_star_schema.db')
cursor = conn.cursor()



In [51]:
table_creation_queries = [
    """
    CREATE TABLE IF NOT EXISTS PlayerDim (
        PlayerID INTEGER PRIMARY KEY,
        Name TEXT,
        Age INTEGER,
        Height_cm INTEGER,
        Weight_kg INTEGER
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS ClubDim (
        ClubID INTEGER PRIMARY KEY,
        ClubName TEXT,
        League TEXT
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS NationalityDim (
        NationalityID INTEGER PRIMARY KEY,
        Nationality TEXT
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS PositionDim (
        PositionID INTEGER PRIMARY KEY,
        Position TEXT
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS TimeDim (
        TimeID INTEGER PRIMARY KEY,
        Year INTEGER,
        Month INTEGER
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS PlayerPerformance (
        PlayerID INTEGER,
        ClubID INTEGER,
        NationalityID INTEGER,
        PositionID INTEGER,
        TimeID INTEGER,
        Overall INTEGER,
        Wage_EUR INTEGER,
        FOREIGN KEY (PlayerID) REFERENCES PlayerDim(PlayerID),
        FOREIGN KEY (ClubID) REFERENCES ClubDim(ClubID),
        FOREIGN KEY (NationalityID) REFERENCES NationalityDim(NationalityID),
        FOREIGN KEY (PositionID) REFERENCES PositionDim(PositionID),
        FOREIGN KEY (TimeID) REFERENCES TimeDim(TimeID)
    )
    """
]

for query in table_creation_queries:
    cursor.execute(query)

conn.commit()


In [53]:

tables = {
    'PlayerDim': pd.read_csv('PlayerDim.csv').rename(columns={
        'Height(cm)': 'Height_cm', 'Weight(kg)': 'Weight_kg'
    }),
    'ClubDim': pd.read_csv('ClubDim.csv').rename(columns={
        'Club': 'ClubName'
    }),
    'NationalityDim': pd.read_csv('NationalityDim.csv'),
    'PositionDim': pd.read_csv('PositionDim.csv').rename(columns={
        'PrimaryPosition': 'Position'
    }),
    'TimeDim': pd.read_csv('TimeDim.csv').drop(columns=['Contract'], errors='ignore'),
    'PlayerPerformance': pd.read_csv('PlayerPerformance.csv').rename(columns={
        'OverallRating': 'Overall', 'Wages': 'Wage_EUR'
    })
}

In [59]:
tables['PlayerDim'] = tables['PlayerDim'].drop_duplicates(subset='PlayerID')
tables['ClubDim'] = tables['ClubDim'].drop_duplicates(subset='ClubID')
tables['NationalityDim'] = tables['NationalityDim'].drop_duplicates(subset='NationalityID')
tables['PositionDim'] = tables['PositionDim'].drop_duplicates(subset='PositionID')
tables['TimeDim'] = tables['TimeDim'].drop_duplicates(subset='TimeID')

In [61]:
for table_name, df in tables.items():
    df.to_sql(table_name, conn, if_exists='replace', index=False)

# Validate the Load

In [66]:
for table in tables.keys():
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    count = cursor.fetchone()[0]
    print(f"{table} has {count} rows.")


PlayerDim has 18891 rows.
ClubDim has 682 rows.
NationalityDim has 164 rows.
PositionDim has 15 rows.
TimeDim has 1 rows.
PlayerPerformance has 18979 rows.


# Foreign Key Integrity

In [68]:
checks = [
    ("PlayerID", "PlayerDim"),
    ("ClubID", "ClubDim"),
    ("NationalityID", "NationalityDim"),
    ("PositionID", "PositionDim"),
    ("TimeID", "TimeDim")
]

for fk, dim in checks:
    cursor.execute(f"""
        SELECT COUNT(*) FROM PlayerPerformance 
        WHERE {fk} NOT IN (SELECT {fk} FROM {dim})
    """)
    print(f"Orphaned {fk} in PlayerPerformance: {cursor.fetchone()[0]}")


Orphaned PlayerID in PlayerPerformance: 0
Orphaned ClubID in PlayerPerformance: 0
Orphaned NationalityID in PlayerPerformance: 0
Orphaned PositionID in PlayerPerformance: 0
Orphaned TimeID in PlayerPerformance: 0


In [75]:
df_clubdim = pd.read_sql_query("PRAGMA table_info(ClubDim);", conn)
print(df_clubdim)


   cid      name     type  notnull dflt_value  pk
0    0    ClubID  INTEGER        0       None   0
1    1  ClubName     TEXT        0       None   0
2    2    League     TEXT        0       None   0


In [81]:
query1 = """
SELECT c.ClubName, ROUND(AVG(p.Overall), 2) AS AvgRating
FROM PlayerPerformance p
JOIN ClubDim c ON p.ClubID = c.ClubID
GROUP BY c.ClubName
ORDER BY AvgRating DESC;
"""

df_query1 = pd.read_sql_query(query1, conn)
print(df_query1.head())


                    ClubName  AvgRating
0  \n\n\n\nFC Bayern München      81.48
1        \n\n\n\nReal Madrid      79.77
2              \n\n\n\nInter      79.35
3             \n\n\n\nNapoli      78.69
4            \n\n\n\nChelsea      78.18


In [87]:
query2 = '''
SELECT n.Nationality, pos.Role, ROUND(SUM(p.Wage_EUR), 2) AS TotalWages
FROM PlayerPerformance p
JOIN NationalityDim n ON p.NationalityID = n.NationalityID
JOIN PositionDim pos ON p.PositionID = pos.PositionID
GROUP BY n.Nationality, pos.Role
ORDER BY TotalWages DESC;
'''

df_query2 = pd.read_sql_query(query2, conn)
print(df_query2.head())


  Nationality        Role  TotalWages
0     England    Defender   6052100.0
1       Spain  Midfielder   5936100.0
2     England  Midfielder   5880200.0
3       Spain    Defender   5578250.0
4      Brazil  Midfielder   5558500.0


In [89]:
pd.read_sql_query("PRAGMA table_info(PlayerDim);", conn)


,cid,name,type,notnull,dflt_value,pk
0,0,PlayerID,INTEGER,0,None,0
1,1,Name,TEXT,0,None,0
2,2,Age,INTEGER,0,None,0
3,3,Height_cm,REAL,0,None,0
4,4,Weight_kg,REAL,0,None,0


In [91]:
pd.read_sql_query("PRAGMA table_info(TimeDim);", conn)



,cid,name,type,notnull,dflt_value,pk
0,0,TimeID,INTEGER,0,None,0
1,1,Year,INTEGER,0,None,0
2,2,Month,INTEGER,0,None,0


In [93]:
pd.read_sql_query("PRAGMA table_info(TimeDim);", conn)


,cid,name,type,notnull,dflt_value,pk
0,0,TimeID,INTEGER,0,None,0
1,1,Year,INTEGER,0,None,0
2,2,Month,INTEGER,0,None,0


In [95]:
query3 = """
SELECT pd.Name, pd.Height_cm
FROM PlayerPerformance p
JOIN PlayerDim pd ON p.PlayerID = pd.PlayerID
JOIN TimeDim t ON p.TimeID = t.TimeID
WHERE t.Year = 2024
ORDER BY pd.Height_cm DESC
LIMIT 5;
"""

df_query3 = pd.read_sql_query(query3, conn)
print(df_query3)


Empty DataFrame
Columns: [Name, Height_cm]
Index: []


In [97]:
query4 = """
SELECT t.Year, pos.Role, ROUND(AVG(p.Overall), 2) AS AvgOverall
FROM PlayerPerformance p
JOIN TimeDim t ON p.TimeID = t.TimeID
JOIN PositionDim pos ON p.PositionID = pos.PositionID
GROUP BY t.Year, pos.Role
ORDER BY t.Year, AvgOverall DESC;
"""

df_query4 = pd.read_sql_query(query4, conn)
print(df_query4)


   Year        Role  AvgOverall
0  2021    Defender       65.94
1  2021  Midfielder       65.85
2  2021     Forward       65.68
3  2021  Goalkeeper       64.65


In [99]:
query5 = """
SELECT c.League, 
       ROUND(AVG(p.Wage_EUR / p.Overall), 2) AS WageToRatingRatio
FROM PlayerPerformance p
JOIN ClubDim c ON p.ClubID = c.ClubID
WHERE p.Overall >= 85
GROUP BY c.League
ORDER BY WageToRatingRatio DESC;
"""

df_query5 = pd.read_sql_query(query5, conn)
print(df_query5)


    League  WageToRatingRatio
0  Unknown            1939.01
